# Module 5 Homework - Let's Get Lunch

Suppose we are looking for a good place for lunch on our next trip to La Crosse. Let's collect some data on nearby restaurants.

## <font color="red"> Problem 1 </font>

Goto yelp.com and perform a search with the following parameters

* **Find** Restaurants
* **Near** La Crosse, WI

**Tasks**
1. Copy the resulting web address below and determine the how the     specified search terms related to the resulting address
2. Use requests and Beautiful Soup to download the content of the front page.

https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI
https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&Start=0

In [1]:
# Imports
from composablesoup import find, find_all, get_text, has_attr
from composable.sequence import slice, head, to_list
from composable.strict import map, filter
from composable.string import replace, strip, split
from composable import from_toolz as tlz
import requests
from bs4 import BeautifulSoup
from composable import pipeable
import re
from composablesoup.soup import find_parent, parents, children, find_previous_sibling
from composablesoup.soup import find_previous_sibling, find_previous_siblings, find_next_sibling, find_next_siblings

In [2]:
# Get and process the Yelp search
s = requests.Session() # Start a session
r = s.get('https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&Start=0') # Get a static page
yelp_lacrosse = BeautifulSoup(r.content, "html.parser")

## <font color="red"> Problem 2 </font>

We want to grab the restaurant's name.

1. Use Inspect Element to determine the tags/classes for each of the elements listed above.  
2. Note that all the business information is contained in a`div` that contains a class similar to  `<div class=" ... businessName__09f24__3Wql2 ...">`.  You will need to use a regular expression to match the `businessName` in the class (see lecture 5.3).
3. Write expressions/functions to pull out the name of each restaurant.  
    * Note: The business name is in an unnamed tag, you will need to navigate to the using searches and/or relationship
 
**Confirm that there is an extra restaurant in the list (e.g. 11-12 instead of 10). This is due to an advertisement/sponsered links, which we want to ignore.**

In [3]:
businessName = re.compile('.+')
(yelp_lacrosse
 >> find_all('a', attrs = {'name':businessName})
 >> map(get_text)
)

['Pappi’s Taqueria y Mas',
 'The Waterfront Restaurant & Tavern',
 'Buzzard Billy’s',
 'Restore Public House',
 'Lovechild Restaurant',
 'The Charmant',
 'The Freighthouse Restaurant',
 'Howie’s on La Crosse',
 'The Crow',
 'Piggy’s Restaurant',
 'Schuby’s Neighborhood Butcher']

## <font color="red"> Problem 3 </font>

Since we picked up extra information, we will need to be clever about identifying the information block for each restaurant.  Note that all of the actual search results (but not sponsered links) start with the ranking (e.g. `11.`).  Use the following steps to get a list that contains the information for each restaurant other than the adds.

1. Start by finding the ranking of the restaurant (1., 2., etc.). **Hint:** You will need to use regular expression to match the text of the tag (see lecture 5.3).
2. Now search for a parent of the above tags that surrounds all of the restaurant information.  You will want to use the `find_parent` method on each of the tags from **1.**.  **Hint:** Look through each of the `div` tags that contain the ranking, looking for a meaningful tag name to match with a regular expression.

The resulting list will be the starting point for gathering all of the information.

In [4]:
href = re.compile('/biz/')
name = re.compile('.+')
(yelp_lacrosse
 >> find_all('a', attrs = {'href':href,'name':name})
 >> map(get_text)
)

['The Waterfront Restaurant & Tavern',
 'Buzzard Billy’s',
 'Restore Public House',
 'Lovechild Restaurant',
 'The Charmant',
 'The Freighthouse Restaurant',
 'Howie’s on La Crosse',
 'The Crow',
 'Piggy’s Restaurant',
 'Schuby’s Neighborhood Butcher']

In [5]:
#half of the random garbage I tried trying to do the hw your way
import re
no_ads = pipeable(lambda L: [g if g[0] == '\d+' else list.pop(g) for g in L])
no_ads_2 = pipeable(lambda g: tlz.get('name') if g[0] == ranking else list.pop(g))
no_ads_3 = pipeable(lambda k: k[3] if len(k) == 4 else "removal")
no_None = pipeable(lambda L: [L.pop(g) if g == None else g for g in L])
filter_ads = pipeable(lambda L: list.pop(L) if len(L) < 2 else L[3]) 
length = pipeable(lambda L: len(L))
uno_reverse = pipeable(lambda g: if (has_attr('target')))
ranking = re.compile("'\d'")
(yelp_lacrosse
 >> find_all('a', attrs = {'name':businessName})
 >> map(find_all('a', attrs = {})
 #>> filter(reversed(has_attr('target')))
 #>> map(find_parent)
 #>> map(children)
 #>> uno_reverse
 #>> map(filter_ads)
 #>> map(length)
 #>> filter(has_attr('target')))
 #>> map(find('', text = '1'))
 #>> map(no_ads_3)
 #>> filter(has_attr('class'))
 #>> map(length)
 #>> no_None
 #>> map(tlz.get(3))
 #>> map(tlz.get('name'))
 #>> map(no_blah)
 #>> map(no_ads_2)
 #>> map(no_ads)
 #>> find_all('', text=ranking)
 #>> map(tlz.get(0))
 
)

SyntaxError: invalid syntax (<ipython-input-5-d30eb7d1b7d8>, line 9)

## <font color="red"> Problem 4 </font>

Write expressions/functions to gather each of the following pieces of information for each of the restaurants.

#### Restaurant Name

In [6]:
href = re.compile('/biz/')
name = re.compile('.+')
get_retaurant_names = pipeable(lambda soup: soup
                                            >> find_all('a', attrs = {'href':href,'name':name})
                                            >> map(get_text)
                             )
yelp_lacrosse >> get_retaurant_names

['The Waterfront Restaurant & Tavern',
 'Buzzard Billy’s',
 'Restore Public House',
 'Lovechild Restaurant',
 'The Charmant',
 'The Freighthouse Restaurant',
 'Howie’s on La Crosse',
 'The Crow',
 'Piggy’s Restaurant',
 'Schuby’s Neighborhood Butcher']

#### Rating

In [7]:
#originally I thought you wanted at what spot they were in the numerical list of restaurants
href = re.compile('/biz/')
name = re.compile('.+')
get_first = pipeable(lambda soup: [item[0] for item in soup])
get_ratings_other = pipeable(lambda soup: soup
                                        >> find_all('a', attrs = {'href':href,'name':name})
                                        >> map(find_parent)
                                        >> map(children)
                                        >> get_first
                     )
yelp_lacrosse >> get_ratings_other

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [8]:
(yelp_lacrosse
 >> find_all('a', attrs = {'href':href,'name':name})
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_next_sibling)
 >> map(find('div', attrs = {'aria-label':name,'role':'img'}))
 >> map(tlz.get('aria-label'))
)

['4.5 star rating',
 '4 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4 star rating',
 '4 star rating',
 '3.5 star rating',
 '4 star rating',
 '4.5 star rating']

In [9]:
href = re.compile('/biz/')
name = re.compile('.+')
get_first = pipeable(lambda soup: [item[0] for item in soup])
get_ratings = pipeable(lambda soup: soup
                                         >> find_all('a', attrs = {'href':href,'name':name})
                                         >> map(find_parent)
                                         >> map(find_parent)
                                         >> map(find_parent)
                                         >> map(find_parent)
                                         >> map(find_parent)
                                         >> map(find_next_sibling)
                                         >> map(find('div', attrs = {'aria-label':name,'role':'img'}))
                                         >> map(tlz.get('aria-label'))
                     )
yelp_lacrosse >> get_ratings

['4.5 star rating',
 '4 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4.5 star rating',
 '4 star rating',
 '4 star rating',
 '3.5 star rating',
 '4 star rating',
 '4.5 star rating']

#### Address

In [10]:
#the gross way of doing it
href = re.compile('/biz/')
name = re.compile('.+')
second_item = pipeable(lambda L: [item[1] for item in L])
flatten = pipeable(lambda L: [item for sublist in L for item in sublist])
(yelp_lacrosse
 >> find_all('a', attrs = {'href':href,'name':name})
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_next_sibling)
 >> map(children)
 #>> map(map(get_text))
 >> map(map(children))
 >> map(map(map(find_next_sibling)))
 #>> map(map([:13]) #trying to do slicing or split
 #>> map(map(get_text))
 #>> map(map(map(split(','))))
 #>> map(map(map(second_item)))
 >> map(get_first)
 >> map(map(get_text))
 >> flatten
)

['328 Front St S',
 '222 Pearl St',
 '1810 State St',
 '300 3rd St S',
 '101 State St',
 '107 Vine St',
 '1128 La Crosse St',
 '100 3rd St S',
 '501 Front St S',
 '321 State St']

In [11]:
#the slightly less gross way of doing it
(yelp_lacrosse
 >> find_all('a', attrs = {'href':href,'name':name})
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_next_sibling)
 >> map(find('span'))
 #>> map(get_first)
 >> map(get_text)
 #>> flatten
)

['328 Front St S',
 '222 Pearl St',
 '1810 State St',
 '300 3rd St S',
 '101 State St',
 '107 Vine St',
 '1128 La Crosse St',
 '100 3rd St S',
 '501 Front St S',
 '321 State St']

In [12]:
href = re.compile('/biz/')
name = re.compile('.+')
get_addresses = pipeable(lambda soup: soup
                                         >> find_all('a', attrs = {'href':href,'name':name})
                                         >> map(find_parent)
                                         >> map(find_parent)
                                         >> map(find_parent)
                                         >> map(find_parent)
                                         >> map(find_parent)
                                         >> map(find_parent)
                                         >> map(find_parent)
                                         >> map(find_next_sibling)
                                         >> map(find('span'))
                                         >> map(get_text)
                        )
yelp_lacrosse >> get_addresses

['328 Front St S',
 '222 Pearl St',
 '1810 State St',
 '300 3rd St S',
 '101 State St',
 '107 Vine St',
 '1128 La Crosse St',
 '100 3rd St S',
 '501 Front St S',
 '321 State St']

#### Review Count

In [13]:
(yelp_lacrosse
 >> find_all('a', attrs = {'href':href,'name':name})
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_next_sibling)
 #>> map(children)
 #>> map(children)
 #>> map(children)
 #>> map(find_next_sibling)
 >> map(map(get_text))
 >> flatten
)

['227', '277', '17', '109', '149', '125', '57', '154', '131', '17']

In [14]:
href = re.compile('/biz/')
name = re.compile('.+')
get_first = pipeable(lambda soup: [item[0] for item in soup])
flatten = pipeable(lambda L: [item for sublist in L for item in sublist])
get_review_count = pipeable(lambda soup: soup
                                             >> find_all('a', attrs = {'href':href,'name':name})
                                             >> map(find_parent)
                                             >> map(find_parent)
                                             >> map(find_parent)
                                             >> map(find_parent)
                                             >> map(find_parent)
                                             >> map(find_next_sibling)
                                             >> map(map(get_text))
                                             >> flatten
                           )
yelp_lacrosse >> get_review_count

['227', '277', '17', '109', '149', '125', '57', '154', '131', '17']

#### Category

**Examples:** `['American (New)', 'Seafood', 'Steakhouses']` becomes `'American (New);Seafood;Steakhouses'`

In [15]:
(yelp_lacrosse
 >> find_all('a', attrs = {'href':href,'name':name})
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_parent)
 >> map(find_next_sibling)
 >> map(find_next_sibling)
 #>> map(children)
 #>> map(children)
 #>> map(children)
 #>> map(find_next_sibling)
 >> map(map(get_text))
 #>> flatten
 >> map(map(replace('$','')))
 >> map(map(replace(', ',';')))
 >> flatten
)

['American (New);Seafood;Steakhouses',
 'American (Traditional);Cajun/Creole',
 'American (Traditional)',
 'American (New)',
 'French;Cocktail Bars',
 'Seafood;Steakhouses;Desserts',
 'American (New);Pubs',
 'American (Traditional);Gastropubs',
 'Steakhouses;Seafood;Sandwiches',
 'Butcher;Delis;Caterers']

In [16]:
href = re.compile('/biz/')
name = re.compile('.+')
get_category = pipeable(lambda soup: soup
                                     >> find_all('a', attrs = {'href':href,'name':name})
                                     >> map(find_parent)
                                     >> map(find_parent)
                                     >> map(find_parent)
                                     >> map(find_parent)
                                     >> map(find_parent)
                                     >> map(find_next_sibling)
                                     >> map(find_next_sibling)
                                     >> map(map(get_text))
                                     >> map(map(replace('$',''))) 
                                     >> map(map(replace(', ',';')))
                                     >> flatten
                       )
yelp_lacrosse >> get_category

['American (New);Seafood;Steakhouses',
 'American (Traditional);Cajun/Creole',
 'American (Traditional)',
 'American (New)',
 'French;Cocktail Bars',
 'Seafood;Steakhouses;Desserts',
 'American (New);Pubs',
 'American (Traditional);Gastropubs',
 'Steakhouses;Seafood;Sandwiches',
 'Butcher;Delis;Caterers']

In [17]:
get_category = pipeable(lambda soup: soup
                                     >> find_all('a', attrs = {'href':href,'name':name})
                                     >> map(find_parent)
                                     >> map(find_parent)
                                     >> map(find_parent)
                                     >> map(find_parent)
                                     >> map(find_parent)
                                     >> map(find_next_sibling)
                                     >> map(find_next_sibling)
                                     >> map(map(get_text))
                                     >> map(map(replace('$',''))) 
                                     >> map(map(replace(', ',';')))
                                     >> flatten
                       )
yelp_lacrosse >> get_category

['American (New);Seafood;Steakhouses',
 'American (Traditional);Cajun/Creole',
 'American (Traditional)',
 'American (New)',
 'French;Cocktail Bars',
 'Seafood;Steakhouses;Desserts',
 'American (New);Pubs',
 'American (Traditional);Gastropubs',
 'Steakhouses;Seafood;Sandwiches',
 'Butcher;Delis;Caterers']

## <font color="red">  Problem 5 </font>

Package all of the expressions in a function that takes a url as input and returns the table of information.  Use a `def` statement and put the above helper functions in the body of the main function.  Test this function on the front page of the search.

In [18]:
def yelp_restaurant_info(url):
  """gets restaurant info from yelp page excluding ads

     Args: 
        soup: a webpage url

     Returns:
        returns the name, rating, address, review count, and category(s)
  """
  s = requests.Session()
  r = s.get(url)
  soup = BeautifulSoup(r.content, 'lxml')
    
  output = list(zip(soup >> get_retaurant_names,
                    soup >> get_ratings,
                    soup >> get_addresses,
                    soup >> get_review_count,
                    soup >> get_category
                   ))
 
  return output

In [19]:
yelp_restaurant_info(yelp_lacrosse)

InvalidSchema: No connection adapters were found for '<!DOCTYPE html>
<html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/no-js/,"js");</script><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="en-US" http-equiv="Content-Language"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><link content="#c41200" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/97ea654d5227/assets/img/logos/yelp_burst.svg" rel="mask-icon" sizes="any"/><link href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/118ff475a341/assets/img/logos/favicon.ico" rel="shortcut icon"/><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><scrip

## <font color="red">  Problem 6 </font>

Now perform a linear search to grab all of the information on restaurants in La Crosse.  You will need to

1. Inspect the url for the first, second, etc. pages to determine the pattern.
2. Create a list of urls for all search results.
3. Get the info from all pages and use your function from the last problem to parse the data into a list.
4. Write the results to a csv file. **Hint:** Use `'a'` (append) instead of `'w'` write on all after the first/

https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&Start=0

In [20]:
page_num = re.compile('text__09f24__2tZKC text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0')
get_third_to_last = pipeable(lambda soup: soup[-3])
get_last = pipeable(lambda soup: soup[-1])
get_int = pipeable(lambda soup: int(soup))
get_total_page_number = pipeable(lambda soup: soup
                                                 >> find_all('span', attrs = {'class':page_num})
                                                 >> map(get_text)
                                                 >> get_third_to_last
                                                 >> split(' ')
                                                 >> get_last
                                                 >> get_int
                        )
yelp_lacrosse >> get_total_page_number

24

In [21]:
soup = yelp_lacrosse
page = [h*10 for h in range(0,(soup >> get_total_page_number))]
page

[0,
 10,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 110,
 120,
 130,
 140,
 150,
 160,
 170,
 180,
 190,
 200,
 210,
 220,
 230]

In [22]:
urls = ['https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&Start=' 
        + str(i) for i in page]
urls

['https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&Start=0',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&Start=10',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&Start=20',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&Start=30',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&Start=40',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&Start=50',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&Start=60',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&Start=70',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&Start=80',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&Start=90',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&

In [23]:
yelp_restaurant_info('https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&Start=150')

[('The Waterfront Restaurant & Tavern',
  '4.5 star rating',
  '328 Front St S',
  '227',
  'American (New);Seafood;Steakhouses'),
 ('Buzzard Billy’s',
  '4 star rating',
  '222 Pearl St',
  '277',
  'American (Traditional);Cajun/Creole'),
 ('Restore Public House',
  '4.5 star rating',
  '1810 State St',
  '17',
  'American (Traditional)'),
 ('Lovechild Restaurant',
  '4.5 star rating',
  '300 3rd St S',
  '109',
  'American (New)'),
 ('The Charmant',
  '4.5 star rating',
  '101 State St',
  '149',
  'French;Cocktail Bars'),
 ('The Freighthouse Restaurant',
  '4 star rating',
  '107 Vine St',
  '125',
  'Seafood;Steakhouses;Desserts'),
 ('Howie’s on La Crosse',
  '4 star rating',
  '1128 La Crosse St',
  '57',
  'American (New);Pubs'),
 ('The Crow',
  '3.5 star rating',
  '100 3rd St S',
  '154',
  'American (Traditional);Gastropubs'),
 ('Piggy’s Restaurant',
  '4 star rating',
  '501 Front St S',
  '131',
  'Steakhouses;Seafood;Sandwiches'),
 ('Schuby’s Neighborhood Butcher',
  '4.5 s

In [ ]:
info_lists = [yelp_restaurant_info(url) for url in urls]
flat_info = [row for info_list in info_lists for row in info_list]
lines = [",".join(row) for row in flat_info]
lines

In [ ]:
with open('yelp_restaurants_lacrosse.csv', 'a') as outfile:
    header = "Restaurant_Name,Rating,Address,Review_Count,Category"
    print(header, file=outfile)
    count = 0
    for url in urls:
        for line in lines:
            count += 1
            if count % 5 == 0:
                print("Processed {0}".format(count))
            print(line, file=outfile)

## <font color="red">  Bonus Problem </font>

See if you can also grab the latitude and longitude of each result.